# Product Matching
## Level 4: Model Training Script

### Notes:
1. Train a Logistic regression model to classify product matches
2. Optimize trained model via hyperparameter tuning

### References: 

## Notebook Config

In [1]:
# Display settings
## Auto reload modules & inline plots
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Package Import & Initialization

In [2]:
# Import packages

import pandas as pd                  #For data manipulation and bgq --> pandas conversion
import numpy as np                   #For scientific computation
import os                            #For work with native operating system and directories
from pathlib import Path             #For working with file paths and directories
import warnings                      #To tweak warning options
import datetime as dt                #For date objects and implemetations
from sklearn.pipeline import Pipeline       #For linking various data ETL moves together
from sklearn.preprocessing import StandardScaler, PolynomialFeatures  #For standardizing/tranforming numeric variables 
from sklearn.linear_model import SGDClassifier       #For training SGD model on data
import joblib                        #For saving objects, more efficient at serializing large numpy arrays than pickle module

In [3]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
warnings.filterwarnings('ignore')

## Function Library

## Custom Tranformers

## Set Directories

In [4]:
#cwd = os.getcwd()
#os.chdir(cwd)
home = str(Path.home())
proj_path = os.path.join(home, 'Cardinal Health', 'Enterprise Data Remediation - Documents', 'General', '01_Projects', 
                         '23_Kinaxis')

In [5]:
main_dir = proj_path + '\\'
iput_dir = main_dir + 'Data\\Output\\ML\\'
oput_dir = main_dir + 'Data\\Output\\ML\\'

## Set Output File Names

In [6]:
## Output file name
mdl_oput_file_name = "L400_mdl_" + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.pkl'

## Load Data

In [7]:
raw_df = pd.read_pickle(iput_dir + "L300_feat_df_2022-07-15.pkl")

In [8]:
X = joblib.load(iput_dir + "L300_prd_df_2022-07-15.pkl")

In [9]:
y = joblib.load(iput_dir + "L300_tgt_df_2022-07-15.pkl")

In [10]:
X_Scaler = joblib.load(iput_dir + "L300_sclr_obj_2022-07-15.pkl")

## High-level Data Inspection

In [11]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1668 entries, 0 to 1667
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ITEMNUMBER              1668 non-null   object 
 1   Number                  1668 non-null   object 
 2   ITEMDESCRIPTION         1668 non-null   object 
 3   FullDescription         1668 non-null   object 
 4   SUPPLIER_ITEM_NUMBER    1668 non-null   object 
 5   ManufacturerItemNumber  1668 non-null   object 
 6   SUPPLIERNAME            1668 non-null   object 
 7   ManufacturerName        1668 non-null   object 
 8   UNSPCCode               1646 non-null   float64
 9   UNSPSC                  1668 non-null   int64  
 10  Feat_Exact_SuppItemNum  1668 non-null   int64  
 11  Feat_Fuzzy_SuppName     1668 non-null   int64  
 12  Feat_Fuzzy_ItemDesc     1668 non-null   int64  
 13  Feat_Exact_ItemNum      1668 non-null   int64  
 14  Feat_Exact_UNSPSC       1668 non-null   

In [12]:
raw_df.head()

,ITEMNUMBER,Number,ITEMDESCRIPTION,FullDescription,SUPPLIER_ITEM_NUMBER,ManufacturerItemNumber,SUPPLIERNAME,ManufacturerName,UNSPCCode,UNSPSC,Feat_Exact_SuppItemNum,Feat_Fuzzy_SuppName,Feat_Fuzzy_ItemDesc,Feat_Exact_ItemNum,Feat_Exact_UNSPSC,Match_Confidence,Match,Comments,Unnamed: 17,Feat_Fuzzy_SuppItemNum
0,58305122,305106,"Regular Bevel Needle 25Gx5/8"" (100 count)",NEEDLE HYPO REGULAR BEVEL 30GX 0.5IN 100/PK 10...,305122,305106,BECTON DICKINSON CONSUMER,BECTON DICKINSON,42142523.0000,42142523,0,100,77,0,1,02 Med,No,NaN,NaN,0
1,683208,A5000-1,"Lisco Nonsterile Sponge 4""x4""",APPLICATOR WOODEN NO TIP 6IN NONSTERILE 1000EA...,3208,A5000-1,CARDINAL HEALTH-PR,CARDINAL HEALTH,42311512.0000,42141502,0,100,53,0,0,03 Low,No,NaN,NaN,0
2,682733,2733,"Curity Nonsterile Gauze Sponge 4""x4""",GAUZE SPONGE 4X4IN 16PLY 2000/CS,2733-,2733-,CARDINAL HEALTH-PR,CARDINAL HEALTH,42311512.0000,42311512,1,100,55,0,1,01 High,Yes,NaN,NaN,1
3,686132,8043,"CURITY Sterile Gauze Pad, 3""x3"", 12 ply",GAUZE SPONGE CURITY 3X3 4PLY LF STER 2EA/PK 25...,6132,8043,CARDINAL HEALTH-PR,CARDINAL HEALTH,42311512.0000,42311512,0,100,57,0,1,03 Low,No,NaN,NaN,0
4,58309642,309642,Luer-Lok Syringe with Detachable PrecisionGlid...,SYRINGE LUER LOK 10ML 21G 1IN 100/PK 4PK/CS,309642,309642,BECTON DICKINSON CONSUMER,BECTON DICKINSON,42142609.0000,42142609,1,100,63,0,1,01 High,Yes,NaN,NaN,1


In [13]:
X.shape

(1668, 6)

In [14]:
y.shape

(1668,)

## Train Model

In [55]:
X_scld = X_Scaler.transform(X)

In [58]:
# sgd_clf = Pipeline([
#     # ("poly_features", PolynomialFeatures(degree=3)),
#     ("scaler", StandardScaler()),
#     ("sgd", SGDClassifier(random_state=42))
# ])

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_scld, y)

SGDClassifier(random_state=42)

## Visualize Model

In [77]:
for name, score in zip(raw_df.filter(regex='^Feat', axis="columns").columns, sgd_clf.coef_.ravel()):
    print(name, score)

Feat_Exact_SuppItemNum 4.12549307593671
Feat_Fuzzy_SuppName 0.7246469433222757
Feat_Fuzzy_ItemDesc 2.172697439648757
Feat_Exact_ItemNum 4.378725658116818
Feat_Exact_UNSPSC 1.1717631762908254
Feat_Fuzzy_SuppItemNum 1.2619895029201347


In [79]:
## Verify a model was created
sgd_clf.predict(X_scld[0:4])

array([0, 0, 1, 0])

## Fine-tune Model (e.g. Grid Search)

In [62]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

Grid Search

In [63]:
gs_param_grid = [{
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
    'loss': ['hinge', 'log'], # logistic regression,
    'penalty': ['l1','elasticnet','l2'],
    'l1_ratio': [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'n_jobs': [-1]
}]

In [64]:
gs_sgd_clf = SGDClassifier(random_state=42)

In [65]:
grid_search = GridSearchCV(gs_sgd_clf, gs_param_grid, cv=5, scoring='f1', return_train_score=True)

In [80]:
grid_search.fit(X_scld, y)

GridSearchCV(cv=5, estimator=SGDClassifier(random_state=42),
             param_grid=[{'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                    1000.0],
                          'l1_ratio': [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
                                       0.8, 0.9],
                          'loss': ['hinge', 'log'], 'n_jobs': [-1],
                          'penalty': ['l1', 'elasticnet', 'l2']}],
             return_train_score=True, scoring='f1')

In [81]:
grid_search.best_params_

{'alpha': 0.001,
 'l1_ratio': 0.1,
 'loss': 'hinge',
 'n_jobs': -1,
 'penalty': 'l1'}

In [82]:
grid_search.best_estimator_

SGDClassifier(alpha=0.001, l1_ratio=0.1, n_jobs=-1, penalty='l1',
              random_state=42)

In [83]:
gs_bst_sgd_clf = grid_search.best_estimator_

Random Search

In [70]:
rs_param_grid = [{
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
    'loss': ['hinge', 'log'], # logistic regression,
    'penalty': ['l1','elasticnet','l2'],
    'l1_ratio': [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'n_jobs': [-1]
}]

In [71]:
rs_sgd_clf = SGDClassifier(random_state=42)

In [87]:
rand_search = RandomizedSearchCV(rs_sgd_clf, rs_param_grid, n_iter=10000, cv=5, scoring='f1', return_train_score=True)

In [88]:
rand_search.fit(X_scld, y)

RandomizedSearchCV(cv=5, estimator=SGDClassifier(random_state=42), n_iter=10000,
                   param_distributions=[{'alpha': [0.0001, 0.001, 0.01, 0.1,
                                                   1.0, 10.0, 100.0, 1000.0],
                                         'l1_ratio': [0.1, 0.15, 0.2, 0.3, 0.4,
                                                      0.5, 0.6, 0.7, 0.8, 0.9],
                                         'loss': ['hinge', 'log'],
                                         'n_jobs': [-1],
                                         'penalty': ['l1', 'elasticnet',
                                                     'l2']}],
                   return_train_score=True, scoring='f1')

In [89]:
rand_search.best_params_

{'penalty': 'l1',
 'n_jobs': -1,
 'loss': 'hinge',
 'l1_ratio': 0.1,
 'alpha': 0.001}

In [86]:
rs_bst_sgd_clf = rand_search.best_estimator_

## Save Model to Modeling Environment

In [78]:
joblib.dump(rs_bst_sgd_clf, oput_dir + mdl_oput_file_name)

['C:\\Users\\kehinde.salau\\Cardinal Health\\Enterprise Data Remediation - Documents\\General\\01_Projects\\23_Kinaxis\\Data\\Output\\ML\\L400_mdl_2022-08-01.pkl']